In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
import segmentation_models_pytorch as smp
from torch.utils.data import DataLoader
from torchvision import transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import ttest_ind, f_oneway
from skimage.metrics import hausdorff_distance


In [ ]:

model = smp.Unet(
    encoder_name="efficientnet-b7",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1
)


In [ ]:

dtype = torch.float16 if torch.cuda.is_available() else torch.float32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device, dtype=dtype)


In [ ]:

criterion = smp.losses.DiceLoss(mode='binary') + nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3)


In [ ]:

transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])


In [ ]:

def predict(model, image):
    model.eval()
    with torch.no_grad():
        image = transform(image=image)['image'].unsqueeze(0).to(device, dtype=dtype)
        output = model(image)
        return torch.sigmoid(output).cpu().numpy()


In [ ]:

def visualize_segmentation(image, mask, prediction):
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    ax[0].imshow(image)
    ax[0].set_title("Original Image")
    ax[1].imshow(mask, cmap='gray')
    ax[1].set_title("Ground Truth Mask")
    ax[2].imshow(prediction, cmap='jet', alpha=0.6)
    ax[2].set_title("Predicted Mask")
    plt.show()


In [ ]:

def compute_statistics(predictions, ground_truths):
    dice_scores = [2 * np.sum(p * g) / (np.sum(p) + np.sum(g) + 1e-7) for p, g in zip(predictions, ground_truths)]
    hausdorff_distances = [hausdorff_distance(p, g) for p, g in zip(predictions, ground_truths)]
    t_test = ttest_ind(dice_scores, hausdorff_distances)
    anova = f_oneway(dice_scores, hausdorff_distances)
    print(f"Mean Dice Score: {np.mean(dice_scores):.4f}")
    print(f"Mean Hausdorff Distance: {np.mean(hausdorff_distances):.4f}")
    print(f"T-Test p-value: {t_test.pvalue:.4f}")
    print(f"ANOVA p-value: {anova.pvalue:.4f}")


In [ ]:

print("Model optimized with EfficientNetB7, visualization, and statistical analysis!")
